In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
! git clone https://github.com/huggingface/transformers.git
! pip3 install av
    
import os
import random
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import shutil
import av
import sys
import torch
from tqdm import tqdm
from time import process_time

from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from transformers import VivitConfig, VivitModel, VivitImageProcessor, VivitForVideoClassification

Cloning into 'transformers'...
remote: Enumerating objects: 233299, done.
remote: Counting objects: 100% (22034/22034), done.
remote: Compressing objects: 100% (1478/1478), done.
remote: Total 233299 (delta 21479), reused 20629 (delta 20464), pack-reused 211265 (from 1)
Receiving objects: 100% (233299/233299), 239.21 MiB | 25.35 MiB/s, done.
Resolving deltas: 100% (170829/170829), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 47.6 MB/s eta 0:00:00


In [3]:
data_dir = '../input/hand-wash-dataset/HandWashDataset/HandWashDataset'

classes = ['Step_1', 'Step_2_Left', 'Step_2_Right', 'Step_3', 'Step_4_Left', 'Step_4_Right', 'Step_5_Left',
           'Step_5_Right', 'Step_6_Left', 'Step_6_Right', 'Step_7_Left', 'Step_7_Right']

def get_classes_filenames(data_dir, classes):
    """
    Filenames of files in data_dir/classes[i] for each i
    Args:
        data_dir (`str`): filepath to a directory
        classes (`list[str]`): list of subdirectories of data_dir
    Return:
        filenames (`dict`): dictionary of filenames
    """
    filenames = {}
    for class_name in classes:
        class_dir = os.path.join(data_dir, class_name)
        filenames[class_name] = os.listdir(class_dir)
    return filenames

filenames = get_classes_filenames(data_dir, classes)

In [4]:
# Auxilliary functions from Hugging Face

def read_video_pyav(container, indices):
    '''
    Decode the video with PyAV decoder.
    Args:
        container (`av.container.input.InputContainer`): PyAV container.
        indices (`List[int]`): List of frame indices to decode.
    Returns:
        result (np.ndarray): np array of decoded frames of shape (num_frames, height, width, 3).
    '''
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])

def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
    '''
    Sample a given number of frame indices from the video.
    Args:
        clip_len (`int`): Total number of frames to sample.
        frame_sample_rate (`int`): Sample every n-th frame.
        seg_len (`int`): Maximum allowed index of sample's last frame.
    Returns:
        indices (`List[int]`): List of sampled frame indices
    '''
    converted_len = int(clip_len * frame_sample_rate)
    end_idx = np.random.randint(converted_len, seg_len)
    start_idx = end_idx - converted_len
    indices = np.linspace(start_idx, end_idx, num=clip_len)
    indices = np.clip(indices, start_idx, end_idx - 1).astype(np.int64)
    return indices

label2id = {class_name: idx for idx, class_name in enumerate(classes)}
id2label = {idx : class_name for idx, class_name in enumerate(classes)}

# Class of dataset
class HandwashingDataset(Dataset):
    """Handwashing Dataset."""
    def __init__(self, file_path, file_names, image_processor, transform=None):
        self.labels = []
        self.file_paths = []
        self.image_processor = image_processor
        for label in file_names.keys():
            for name in file_names[label]:
                full_path = os.path.join(file_path, label, name)
                self.file_paths.append(full_path)
                self.labels.append(label2id[label])
        
    def __len__(self):
        return len(self.file_paths)
    
    def __getitem__(self, idx):
        #t1 = process_time()
        vid_container = av.open(self.file_paths[idx])
        #t2 = process_time()
        vid_indices = sample_frame_indices(clip_len=32, frame_sample_rate=1, seg_len=vid_container.streams.video[0].frames)
        #t3 = process_time()
        vid_read = read_video_pyav(container=vid_container, indices=vid_indices)
        #t4 = process_time()
        vid_input = self.image_processor(list(vid_read), return_tensors="pt")
        #t5 = process_time()
        vid_input['pixel_values'] = vid_input['pixel_values'].squeeze(0)
        #print(f"av.open takes:       {t2 - t1}")
        #print(f"sample_frame takes:  {t3 - t2}")
        #print(f"read_video takes:    {t4 - t3}")
        #print(f"img_process takes:   {t5 - t4}")
        
        return {'video' : vid_input, 'class' : torch.tensor(self.labels[idx], dtype=torch.long)}

In [5]:
def split_dataset(filenames, test_size=0.2, random_state=20):
    test = {}
    train = {}
    for label in filenames.keys():
        train_names, test_names = train_test_split(filenames[label], test_size=test_size, random_state=random_state)
        test[label] = test_names
        train[label] = train_names
    return train, test

# Preparing the dataset and splititng into training and test datasets
train_vid_names, test_vid_names = split_dataset(filenames, random_state=8)

In [6]:
def train_vivit_model(model, data_loader, criterion, optimizer, device, num_epochs=10):
    """
    Trains the Vision Transformer (ViViT) model.
    
    Args:
        model (`nn.Module`): The ViViT model to train.
        data_loader (`DataLoader`): The PyTorch DataLoader with HandwashingDataset.
        criterion (`torch.nn.Module`): Loss function (e.g., CrossEntropyLoss).
        optimizer (`torch.optim.Optimizer`): Optimizer (e.g., Adam, SGD).
        device (`torch.device`): Device to train on ('cuda' or 'cpu').
        num_epochs (`int`): Number of epochs to train (default: 10).
    
    Returns:
        model: Trained ViViT model.
    """
    # Move model to the device (GPU or CPU)
    model.to(device)
    
    # Training loop
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        running_loss = 0.0
        correct_predictions = 0
        total_samples = 0
        
        # Loop over data batches
        for batch in tqdm(data_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            # Get inputs and labels, move them to the device
            inputs = batch['video'].to(device)  # (batch_size, num_frames, C, H, W)
            labels = batch['class'].to(device)  # (batch_size,)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(**inputs)  # (batch_size, num_classes)
            
            # Calculate the loss
            loss = criterion(outputs.logits, labels)
            
            # Backward pass + optimize
            loss.backward()
            optimizer.step()
            
            # Track loss and accuracy
            running_loss += loss.item()
            preds = outputs.logits.argmax(-1).item()  # Get predicted class
            correct_predictions += (preds == labels).sum().item()
            total_samples += labels.size(0)
        
        # Calculate average loss and accuracy for the epoch
        epoch_loss = running_loss / total_samples
        epoch_acc = correct_predictions / total_samples
        
        # Print statistics
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')
    
    return model

In [7]:
"""
class VideoPreprocessor(VivitImageProcessor):
    def __init__(self, **kwargs):
        super.__init__(**kwargs)
        
    def process(self, image):
"""      

'\nclass VideoPreprocessor(VivitImageProcessor):\n    def __init__(self, **kwargs):\n        super.__init__(**kwargs)\n        \n    def process(self, image):\n'

In [8]:
# Shrink the size of the model
config = VivitConfig(hidden_size=256,num_hidden_layers=4,intermediate_size=768,num_attention_heads=4)

config.num_labels = len(classes)
for i in range(len(classes)):
    id2label[i] = classes[i]
    label2id[classes[i]] = i
config.id2label = id2label
config.label2id = label2id

In [9]:
# create the model from the config
model = VivitForVideoClassification(config)
#model = torch.load("cur_model.pkl", weights_only=False)

# set the optimizer, criterion, and device
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available else nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# input the data
# image_processor = VivitImageProcessor.from_pretrained("google/vivit-b-16x2-kinetics400")
image_processor = VivitImageProcessor()
dataset = HandwashingDataset(data_dir, train_vid_names, image_processor)
data_loader = DataLoader(dataset, batch_size = 1, shuffle=True)

train_vivit_model(model, data_loader, criterion, optimizer, device, 20)

Epoch 1/20:   0%|          | 0/240 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/feature_extraction_utils.py:142: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:278.)
  return torch.tensor(value)
Epoch 1/20: 100%|██████████| 240/240 [09:29<00:00,  2.37s/it]


Epoch [1/20], Loss: 2.8049, Accuracy: 0.0667


Epoch 2/20: 100%|██████████| 240/240 [09:37<00:00,  2.41s/it]


Epoch [2/20], Loss: 2.6161, Accuracy: 0.0750


Epoch 3/20: 100%|██████████| 240/240 [09:35<00:00,  2.40s/it]


...

Epoch 19/20: 100%|██████████| 240/240 [09:35<00:00,  2.40s/it]


Epoch [19/20], Loss: 2.4945, Accuracy: 0.0750


Epoch 20/20: 100%|██████████| 240/240 [09:37<00:00,  2.40s/it]

Epoch [20/20], Loss: 2.4993, Accuracy: 0.0583


VivitForVideoClassification(
  (vivit): VivitModel(
    (embeddings): VivitEmbeddings(
      (patch_embeddings): VivitTubeletEmbeddings(
        (projection): Conv3d(3, 256, kernel_size=(2, 16, 16), stride=(2, 16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): VivitEncoder(
      (layer): ModuleList(
        (0-3): 4 x VivitLayer(
          (attention): VivitAttention(
            (attention): VivitSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): VivitSelfOutput(
              (dense): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): VivitIntermediate(
            (

In [10]:
torch.save(model, "cur_model.pkl")

<a href="cur_model.pkl"> Download File </a>